In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import sys
import torch  
import gym
import queue
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import math

FLOAT_BYTE_SIZE = 8
HOST_CPU_FLOP = 62000000000
MAX_MEMORY = 0
MEMORY_LEVELS = 30
INVALID = -1
INVALID_INCR = -0.0001
BETA = 0.5
GAMMA = 0.9
WORKER_MEM = 165

MAX_OP_PARAM = []
MAX_ED_PARAM = []

def init():
    for i in range(5):
        MAX_OP_PARAM.append(0.0)
    for i in range(5):
        MAX_ED_PARAM.append(0.0)


init()

def calc_level(mem):
    #global MAX_MEMORY, MEMORY_LEVELS
    interval = MAX_MEMORY/MEMORY_LEVELS
    
    if(mem<=0):
            return 0

    return int(mem/interval)+1

In [2]:
class Computation_Graph:
    
    def __init__(self, filename, memoryfile):
        
        #batch size
        #number of layers, number of connections
        #for each layer
            #layer name, layer_mem in MB, # of params in layer,  dimension2, dimensioon3, ...

        fileReader = open(filename,'r')
        memoryfile = open(memoryfile,'r')

        #self.batch_size = int(fileReader.readline())
        self.batch_size = 1

        vertex, edge = (fileReader.readline().split())
        self.num_layers = int(vertex)
        self.num_connections = int(edge)

        self.layer_estimated_mem = []
        self.layer_estimated_float_op = [] 
        self.layer_output_shape = []
        self.layer_name_to_idx_map = {}
        self.layer_idx_to_name_map = {}
        self.adj_list = []
        self.rev_adj_list = []

        for i in range(self.num_layers):
            
            print(i)
            self.adj_list.append([])
            self.rev_adj_list.append([])
            
            layer_info = fileReader.readline().split() #layer_info is an array now
            memory_info = memoryfile.readline().split()
            
            print(memory_info)
            
            self.layer_name_to_idx_map[layer_info[0]] = i
            self.layer_idx_to_name_map[i] = layer_info[0]

            self.layer_estimated_mem.append(float(memory_info[1])-WORKER_MEM)
            global MAX_MEMORY
            MAX_MEMORY = max(MAX_MEMORY,float(memory_info[1]))

            self.layer_estimated_float_op.append(float(layer_info[1]))
            
            #this has to be rewritten
            output_byte = self.batch_size*FLOAT_BYTE_SIZE #floating point number is 4 byte

            #for k in range(3,len(layer_info)):
                #output_byte = output_byte*(float(layer_info[k]))

            #self.layer_output_shape.append(float(output_byte))
            self.layer_output_shape.append(0.0)
            #this has to be rewritten

        for i in range(self.num_connections):
            #this has to be rewritten
            u,v,w = fileReader.readline().split()
            self.adj_list[self.layer_name_to_idx_map[u]].append(self.layer_name_to_idx_map[v])
            self.rev_adj_list[self.layer_name_to_idx_map[v]].append(self.layer_name_to_idx_map[u])
            self.layer_output_shape[self.layer_name_to_idx_map[u]] = float(w)

        #print(type(self.adj_list[0][0]))
        #print(self.adj_list)
        fileReader.close()

    def print_info(self):
        print(self.layer_estimated_mem)
        print(self.layer_estimated_float_op)
        print(self.layer_output_shape) 
        
    def get_mem_required(self, layer_name, MB = 1):

        val = self.layer_estimated_mem[self.layer_name_to_idx_map[layer_name]]
        
        if MB == 0:
            return (float(val))*(1024.0*1024.0)

        return val

    def get_mem_level(self, layer_idx):

        global MAX_MEMORY,MEMORY_LEVELS

        interval = MAX_MEMORY/MEMORY_LEVELS

        if(self.layer_estimated_mem[layer_idx]<=0):
            return 0

        return int(self.layer_estimated_mem[layer_idx]/interval)+1


    def get_cpu_required(self, layer_name):

        return self.layer_estimated_float_op[self.layer_name_to_idx_map[layer_name]]

    def get_data_transferred(self, layer_u, layer_v):

        u = self.layer_name_to_idx_map[layer_u]
        v = self.layer_name_to_idx_map[layer_v]

        if v in self.adj_list[u]:
            return self.layer_output_shape[u]

        return 0

    def toposort(self):
        
        topoReader = open("topo_out.txt")
        topo_order = []

        for i in range(self.num_layers):
            v, level = topoReader.readline().split()
            topo_order.append((int(v),int(level)))

        return (1,topo_order)

    def check_topo_sort(self):

        acyclic,topo = self.toposort()

        if(acyclic==1):

            for v in topo:
                print(self.layer_idx_to_name_map[v[0]])

        else:
            print('no topo exists')


#sample input for testing one correctness
# 30
# 3 3
# conv_1 20 10 28 28 1
# conv_2 10 20 32 32 1
# dense_1 100 10 128 128
# conv_2 dense_1
# conv_1 dense_1
# conv_1 conv_2


In [3]:
class Device_Graph:
    
    def __init__(self,filename):
        
        #num_devices.. where devices connected in star topology
        #name, num_flop, memory in MB

        fileReader = open(filename,'r')
        
        device_no = int(fileReader.readline())
        
        self.device_memory = []
        self.device_flops = [] 
        self.device_name_to_idx_map = {}
        self.device_idx_to_name_map = {}
        self.bridge_bandwidth = [] #bandwidth to bridge

        for i in range(device_no):
            
            edge_info = fileReader.readline().split()
            
            self.device_memory.append(float(edge_info[2])-WORKER_MEM)
            global MAX_MEMORY
            MAX_MEMORY = max(MAX_MEMORY,float(edge_info[2]))

            self.device_flops.append(float(edge_info[1]))
            
            self.device_name_to_idx_map[edge_info[0]] = i
            self.device_idx_to_name_map[i] = edge_info[0]

        for i in range(device_no):
            bw_master = float(fileReader.readline())
            self.bridge_bandwidth.append(bw_master)
        
        #print(self.bridge_bandwidth)

    def get_mem_level(self, device_idx):

        interval = MAX_MEMORY/MEMORY_LEVELS

        if(self.device_memory[device_idx]<=0):
            return 0

        return int(self.device_memory[device_idx]/interval)+1

    def get_device_flop(self, device_idx):

        return self.device_flops[device_idx]

    def print_info(self):
        print(self.device_memory)
        print(self.device_flops)
        print(self.bridge_bandwidth)

In [25]:
class Environment:
    
    def __init__(self, comp_graph, device_graph):
        
        self.comp_graph, self.device_graph = comp_graph, device_graph 
        self.current_state = np.array([],np.float)
        self.has_topo, self.topo_order = self.comp_graph.toposort()
        print(self.topo_order)
        self.available_mem = self.device_graph.device_memory
        self.placement_dict = {}
        
    def get_layer_tuple(self, layer_idx): #this idx is topo_sort[idx], from this we get 
        #layer_idx
        #flop required
        #data transfer
        #required memory level
        ##number of neighbors
        return [layer_idx,self.comp_graph.layer_estimated_float_op[layer_idx],self.comp_graph.layer_output_shape[layer_idx],self.comp_graph.get_mem_level(layer_idx)]

    def get_edge_tuple(self, device_idx):

        #current memory level of device
        #number of operations incorporated
        #bandwidth to bridge
        #flops power
        ##number of floating point operations performed
        return [self.device_graph.get_mem_level(device_idx),0,self.device_graph.bridge_bandwidth[device_idx],self.device_graph.device_flops[device_idx]]


    def print_current_state(self):
        #state e ektai layer ... [0][1] to [0][3]
        print("will place "+str(self.current_state[0][0])+" now",end=' ')
        print(", requires "+str(self.current_state[0][1])+" flops",end=' ')
        print(", will tranx "+str(self.current_state[0][2])+" data",end=' ')
        print(", requires "+str(self.current_state[0][3])+" level memory")
        
        #device related info

        for i in range(1,1+len(self.device_graph.device_memory)):
            print("device "+str(i-1),end=' ')
            print(", has memory level "+str(self.current_state[i][0]),end=' ')
            print(", will exec "+str(self.current_state[i][1])+" operations",end=' ')
            print(", has "+str(self.current_state[i][2])+" bw with bridge",end=' ')
            print(", has "+str(self.current_state[i][3])+" flops")
            
        print('')

        #print(self.current_state)

    def reset(self):
        
        init_state = []
        self.done = 0
        #for i in range(self.comp_graph.num_layers):
        init_state.append(self.get_layer_tuple(self.topo_order[0][0]))

        for i in range(len(self.device_graph.device_memory)):
            init_state.append(self.get_edge_tuple(i))

        np_init_state = np.array(init_state,np.float)

        self.current_state = np_init_state
        self.available_mem = self.device_graph.device_memory.copy() #eita reference e hocche na 

        return self.current_state    

    def is_valid_placement(self): # need further edit
        
        d = len(self.device_graph.device_memory)

        for i in range(1,1+d):
            if(self.current_state[i][0]<=0):
                return 0
        return 1
    
    def evaluate_placement(self): #edit korte hobe
        
        l = self.comp_graph.num_layers
        d = len(self.device_graph.device_memory)

        topo_idx = [] #will contain the index of vertex v in topo_order
        # original vertex list = {0,1,2,3,4}
        # in topo order = {2,1,3,0,4}
        # in topo idx = {3,1,0,2,4}
        for i in range(len(self.topo_order)):
            topo_idx.append(-1)

        for i in range(len(self.topo_order)):
            topo_idx[self.topo_order[i][0]] = i
        
        # if(self.is_valid_placement() == 0):
        #     global INVALID
        #     INVALID = INVALID + INVALID_INCR
        #     return INVALID
        
        estimated_tx_time = 0.0
        
        for i in range(l): #topo_order er serial e jacche
            
            cur_device = int(self.placement_dict[self.topo_order[i][0]]) #because ith placmement directs the placement of the ith vertex in topo_sort
            
            for u in self.comp_graph.adj_list[self.topo_order[i][0]]:
                
                nxt_device =  int(self.placement_dict[u]) #we need to get the index of u from topo_idx
                payload = self.comp_graph.layer_output_shape[self.topo_order[i][0]]
                
                if(cur_device != nxt_device):
                    estimated_tx_time = estimated_tx_time + (payload/self.device_graph.bridge_bandwidth[cur_device]) + (payload/self.device_graph.bridge_bandwidth[nxt_device])  
        
        estimated_ex_time= 0.0
        

        level_wise = []
        #print(self.topo_order)
        #print(level_wise)
        for i in range(len(self.topo_order)+3):
            level_wise.append([])
        
        for i in range(len(self.topo_order)):
            #print(i)
            level_wise[self.topo_order[i][1]].append(self.topo_order[i][0])
        
        for cur_level in range(len(level_wise)):
            
            device_time = []
            
            for i in range(len(self.device_graph.device_memory)):
                device_time.append(0.0)
            
            for i in range(len(level_wise[cur_level])):
                
                cur_v = level_wise[cur_level][i]
                cur_device = self.placement_dict[cur_v]

                flop_req = self.comp_graph.layer_estimated_float_op[cur_v]
                flops_got = self.device_graph.device_flops[cur_device]
            
                device_time[cur_device] = device_time[cur_device]+flop_req/flops_got
            
            estimated_ex_time = estimated_ex_time + max(device_time)

        #estimated_energy_consumption = []

        #for i in range(l,l+d):
        #    estimated_energy_consumption.append(self.current_state[i][4])

        #return (1.0-BETA)*(estimated_ex_time+estimated_tx_time)+BETA*(np.average(estimated_energy_consumption)+np.var(estimated_energy_consumption))
        #return (1.0-BETA)*estimated_tx_time+BETA*estimated_ex_time
        return BETA*estimated_ex_time

    def step(self,action): #action is the index of a device

        l = self.comp_graph.num_layers
        d = len(self.device_graph.device_memory)
        reward = 0.0
        
        layer_processed = int(self.current_state[0][0])
        #current memory level of device
        #number of operations incorporated
        #bandwidth to bridge
        #flops power
        self.available_mem[action] = self.available_mem[action] - self.comp_graph.layer_estimated_mem[layer_processed]
        self.current_state[action+1][0] = calc_level(self.available_mem[action])
        self.current_state[action+1][1] = self.current_state[action+1][1] + 1

        self.done = self.done+1
        self.placement_dict[layer_processed] = action
        
        if self.done==l:

            if self.is_valid_placement()==0:
                return self.current_state,self.placement_dict,-5,1
            else:
                cost = self.evaluate_placement()
                reward = 1.0/math.sqrt(cost)
                #return self.current_state,self.placement_dict,10*reward,1
#                 print("got all right")
#                 print(reward)
                print(f"Finished with correct placement, reward={reward}, evaluation={cost}")
                return self.current_state,self.placement_dict,10*reward,1
        else:
            estimated_tx_time = 0.0
            
            for u in self.comp_graph.rev_adj_list[layer_processed]:
                
                prev_device =  int(self.placement_dict[u]) #we need to get the index of u from topo_idx
                payload = self.comp_graph.layer_output_shape[u]
                
                if(action != prev_device):
                    estimated_tx_time = estimated_tx_time + (payload/self.device_graph.bridge_bandwidth[prev_device]) + (payload/self.device_graph.bridge_bandwidth[action])  
            
            if(estimated_tx_time<=0.0):
                reward = 3
            else:
                reward = 1.0/math.sqrt(estimated_tx_time)
            
            self.current_state[0] = self.get_layer_tuple(self.topo_order[self.done][0])
            
            if (self.available_mem[action]<=0.0):
                return self.current_state,self.placement_dict,-3,0
            else:
                #return self.current_state,self.placement_dict,3*reward,0
                return self.current_state,self.placement_dict,2,0


layer_file = "ml_graph_vgg.txt"
mem_file = "vgg_new_memory_calc.txt"

c = Computation_Graph(layer_file,mem_file)
#c.print_info()

device_file = "dev_graph.txt"
d = Device_Graph(device_file)
#d.print_info()

#c.check_topo_sort()


e = Environment(c,d)

cur_state = e.reset()
e.print_current_state()

for i in range(0,5):
    e.step(i)
    e.print_current_state()
    print(' ')

print(MAX_MEMORY)
print(MEMORY_LEVELS)

# # for i in range(0,18):
# #     e.step(0)
# #     e.print_current_state()
# #     print('')

0
['conv_1', '1153.696']
1
['conv_2', '1929.888']
2
['pool_1', '545.7']
3
['conv_3', '1134.24']
4
['conv_4', '1264.288']
5
['pool_2', '390.9']
6
['conv_5', '654.1']
7
['conv_6', '664.9']
8
['conv_7', '543.1']
9
['pool_3', '363.7']
10
['conv_8', '498.178']
11
['conv_9', '740.3']
12
['conv_10', '740.9']
13
['pool_4', '305.3']
14
['conv_11', '640.4']
15
['conv_12', '695.9']
16
['conv_13', '670.5']
17
['pool_5', '283.9']
18
['flat_1', '200']
19
['dense_1', '630']
20
['dense_2', '670']
21
['dense_3', '324']
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22)]
will place 0.0 now , requires 50176.0 flops , will tranx 50176.0 data , requires 8.0 level memory
device 0 , has memory level 7.0 , will exec 0.0 operations , has 1000.0 bw with bridge , has 1000000000.0 flops
device 1 , has memory level 7.0 , will exec 0.0 operations , has 500.0 bw with b

In [26]:
class PolicyNetwork(nn.Module):
    #num_inputs = size of state vectors == number of operations
    #num_actions = size of edge devices == number of actions
    
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(PolicyNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size) #two layer NN as policy network
        #self.linear3 = nn.Linear(32,16)
        #self.linear4 = nn.Linear(16,16)
        self.linear5 = nn.Linear(hidden_size,hidden_size*2)
        self.linear6 = nn.Linear(hidden_size*2,hidden_size)
        #self.linear7 = nn.Linear(64,32)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        #x = F.relu(self.linear3(x))
        #x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = F.relu(self.linear6(x))
        #x = F.relu(self.linear7(x))
        x = F.softmax(self.linear2(x), dim=1)
        return x 
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        #print(state)
        #print(self.linear1.weight.data)
        #print(self.linear2.weight.data)
        probs = self.forward(Variable(state))
        #print(probs)
        #print(np.squeeze(probs.detach().numpy()))
        highest_prob_action = np.random.choice(self.num_actions, p=np.squeeze(probs.detach().numpy()))
        #print(highest_prob_action)
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action])
        
        return highest_prob_action, log_prob

def update_policy(policy_network, rewards, log_probs):
    
    discounted_rewards = []
    #print(rewards)
    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
    
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9) # normalize discounted rewards
    #print(discounted_rewards)
    
    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    
    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

In [27]:
import copy

def main(placement_environment):
    env = placement_environment
    #v = env.op_graph.toposort()
    layer_num = env.comp_graph.num_layers
    device_num = len(env.device_graph.device_memory)

    inp_size_nn = (1+device_num)*4
    policy_net = PolicyNetwork(inp_size_nn, device_num, 64) #all 3 arguments are int

    
    converge = 0
    max_epoch = 100000
    best_state = {}
    best_reward = 0.0
    best_policy = copy.deepcopy(policy_net)
    
    total_score = 0
    total_count = 0
    ok_count = 0
    
    while converge<max_epoch:
        
        state = env.reset()  #state has been made compatible
        log_probs = []
        rewards = []
        
        if converge%5000==0:
            print(converge)
        
        while True:
            
            temp_state = state.copy()

            for p in range(4):
                temp_state[0][p] = float(temp_state[0][p])/MAX_OP_PARAM[p]

            for d in range(1,1+device_num):
                for p in range(4):
                    temp_state[d][p] = float(temp_state[d][p])/MAX_ED_PARAM[p]

            #network_state = temp_state.flatten()
            #env.print_current_state()
            #inp = (network_state-np.min(network_state))/np.ptp(network_state)
            #inp = network_state
            #val = np.array([np.sum(inp)/2000000000.0,np.sum(inp)/3000000000.0])
            #print(val)
            #print(inp)
            action, log_prob = policy_net.get_action(temp_state.flatten())
            #print(action)
            #print(log_prob)
            #print('action chosen '+str(action))
            new_state, mapping, reward, done = env.step(action)
            #env.print_current_state()
            log_probs.append(log_prob)
            rewards.append(reward)
            
            if done:
                
                #for i in range(layer_num):
                #    print(state[i][0], end = " ")
                #print('')

                converge = converge + 1
                
                score = env.evaluate_placement()
                if reward > 0:
                    total_score += score
                    total_count += 1
                    ok_count += 1
                
                pos = [num for num in rewards if num>0.0]

                if converge%100==0:
#                     print(len(pos))
                    print(f"{converge}: pos={len(pos)}/{len(rewards)}, reward={reward}, evaluation={score}")
                    print(f"%ok: {ok_count / 100}")
                    if total_count > 0:
                        print(f"rolling average score of ok = {total_score / total_count}")
                    if total_count > 100:
                        total_score = 0
                        total_count = 0
                    ok_count = 0
                    print(best_policy.linear1.weight.data)
                
                update_policy(policy_net, rewards, log_probs)
                
                if len(pos) > best_reward:
                    best_reward = len(pos)
                    best_state = mapping
                    best_policy = copy.deepcopy(policy_net)
                    torch.save(best_policy.state_dict(), './mymodel.pt')
                    print("current best state ")
                    print(best_state)                
                #if converge%1000==0:
                #print("state "+str(state))
                #print("reward incurred "+str(reward))
                break
            
            state = new_state
    

      
    return best_state,best_reward, best_policy, inp_size_nn, device_num 

# print("hello")

myenv = e

def set_MAX_PARAMS(environment):
    #layer_idx         
        #flop required
        #data transfer
        #required memory level
    MAX_OP_PARAM[0] = float(len(environment.comp_graph.layer_estimated_mem))
    MAX_OP_PARAM[1] = float(max(environment.comp_graph.layer_estimated_float_op))
    MAX_OP_PARAM[2] = float(max(environment.comp_graph.layer_output_shape))
    MAX_OP_PARAM[3] = float(MEMORY_LEVELS + 1)
    #current memory level of device
        #number of operations incorporated
        #bandwidth to bridge
        #flops power

    MAX_ED_PARAM[0] = float(MEMORY_LEVELS + 1)
    MAX_ED_PARAM[1] = float(environment.comp_graph.num_layers)
    MAX_ED_PARAM[2] = float(max(environment.device_graph.bridge_bandwidth))
    MAX_ED_PARAM[3] = float(max(environment.device_graph.device_flops))

In [28]:
set_MAX_PARAMS(e)

best_state,best_reward, best_policy, inp_size, device_num = main(e)
print("training complete")

print("best state "+str(best_state))
print("estimated computation time "+str(1.0/best_reward**2)+" second")



0
current best state 
{0: 3, 1: 4, 2: 7, 3: 1, 4: 7, 5: 6, 6: 7, 7: 7, 8: 3, 9: 7, 10: 8, 11: 4, 12: 6, 13: 7, 14: 6, 15: 4, 16: 7, 17: 6, 18: 2, 19: 3, 20: 7, 21: 4}
current best state 
{0: 4, 1: 7, 2: 0, 3: 6, 4: 8, 5: 4, 6: 1, 7: 6, 8: 3, 9: 6, 10: 7, 11: 5, 12: 1, 13: 3, 14: 3, 15: 7, 16: 7, 17: 2, 18: 5, 19: 5, 20: 7, 21: 2}
100: pos=18/22, reward=-5, evaluation=0.012196588698412699
%ok: 0.0
tensor([[ 0.0232, -0.0270,  0.0393,  ..., -0.1228, -0.1127,  0.1215],
        [-0.1184, -0.0807, -0.0554,  ...,  0.1201, -0.0889, -0.0270],
        [ 0.0879,  0.0732, -0.0237,  ..., -0.1297, -0.1080,  0.0830],
        ...,
        [ 0.0297, -0.1056,  0.0836,  ...,  0.1178, -0.1212,  0.0496],
        [-0.0378,  0.0840,  0.1113,  ...,  0.1285, -0.1077,  0.0058],
        [ 0.1367,  0.1568, -0.0177,  ...,  0.1136, -0.1193, -0.1286]])
Finished with correct placement, reward=8.635029717357057, evaluation=0.013411344761904762
current best state 
{0: 7, 1: 8, 2: 6, 3: 6, 4: 6, 5: 2, 6: 7, 7: 6, 8: 2, 

Finished with correct placement, reward=7.100190988239362, evaluation=0.019836266666666668
Finished with correct placement, reward=6.453033756697105, evaluation=0.024014421333333334
Finished with correct placement, reward=6.953968555175034, evaluation=0.020679239111111112
Finished with correct placement, reward=5.7646433925920135, evaluation=0.03009228088888889
Finished with correct placement, reward=5.602160193551689, evaluation=0.031863168000000004
Finished with correct placement, reward=6.618086327208312, evaluation=0.022831536761904764
600: pos=22/22, reward=66.18086327208312, evaluation=0.022831536761904764
%ok: 0.13
rolling average score of ok = 0.022145008958525343
tensor([[ 0.0283, -0.0252,  0.0445,  ..., -0.1175, -0.1067,  0.1275],
        [-0.1155, -0.0796, -0.0541,  ...,  0.1192, -0.0868, -0.0249],
        [ 0.0858,  0.0730, -0.0261,  ..., -0.1321, -0.1109,  0.0802],
        ...,
        [ 0.0297, -0.1056,  0.0836,  ...,  0.1178, -0.1212,  0.0496],
        [-0.0336,  0.0863,

Finished with correct placement, reward=6.574170451200347, evaluation=0.02313758780952381
Finished with correct placement, reward=8.06276321492931, evaluation=0.015382686476190476
Finished with correct placement, reward=5.7601604236628665, evaluation=0.03013913904761905
Finished with correct placement, reward=6.338818617083043, evaluation=0.02488761904761905
1000: pos=17/22, reward=-5, evaluation=0.020695405714285717
%ok: 0.16
rolling average score of ok = 0.023754510952380948
tensor([[ 0.0283, -0.0252,  0.0445,  ..., -0.1175, -0.1067,  0.1275],
        [-0.1155, -0.0796, -0.0541,  ...,  0.1192, -0.0868, -0.0249],
        [ 0.0858,  0.0730, -0.0261,  ..., -0.1321, -0.1109,  0.0802],
        ...,
        [ 0.0297, -0.1056,  0.0836,  ...,  0.1178, -0.1212,  0.0496],
        [-0.0336,  0.0863,  0.1104,  ...,  0.1329, -0.1047,  0.0087],
        [ 0.1369,  0.1569, -0.0164,  ...,  0.1131, -0.1186, -0.1278]])
Finished with correct placement, reward=6.57321943180298, evaluation=0.0231442834285

Finished with correct placement, reward=7.2034911674844775, evaluation=0.019271430095238094
Finished with correct placement, reward=6.4542637478340135, evaluation=0.024005269333333336
Finished with correct placement, reward=6.577407961383933, evaluation=0.023114816
1400: pos=14/22, reward=-5, evaluation=0.019789549714285717
%ok: 0.18
rolling average score of ok = 0.022969739738288808
tensor([[ 0.0283, -0.0252,  0.0445,  ..., -0.1175, -0.1067,  0.1275],
        [-0.1155, -0.0796, -0.0541,  ...,  0.1192, -0.0868, -0.0249],
        [ 0.0858,  0.0730, -0.0261,  ..., -0.1321, -0.1109,  0.0802],
        ...,
        [ 0.0297, -0.1056,  0.0836,  ...,  0.1178, -0.1212,  0.0496],
        [-0.0336,  0.0863,  0.1104,  ...,  0.1329, -0.1047,  0.0087],
        [ 0.1369,  0.1569, -0.0164,  ...,  0.1131, -0.1186, -0.1278]])
Finished with correct placement, reward=6.462495685400824, evaluation=0.02394415238095238
Finished with correct placement, reward=6.958342282135429, evaluation=0.02065325104761905

Finished with correct placement, reward=5.681776535733904, evaluation=0.030976454095238096
Finished with correct placement, reward=7.680800263001721, evaluation=0.01695067733333333
Finished with correct placement, reward=5.7687674334156185, evaluation=0.030049270857142856
Finished with correct placement, reward=6.344356054592293, evaluation=0.024844193523809524
Finished with correct placement, reward=9.764905566296912, evaluation=0.010487305142857143
Finished with correct placement, reward=9.644153916057501, evaluation=0.010751566222222222
Finished with correct placement, reward=7.374967528733301, evaluation=0.018385683301587304
Finished with correct placement, reward=8.057515308656011, evaluation=0.015402730666666668
Finished with correct placement, reward=7.869400230634266, evaluation=0.016147925333333334
Finished with correct placement, reward=5.607636271013016, evaluation=0.03180096711111111
Finished with correct placement, reward=8.322045902274736, evaluation=0.014439088761904762


...

Finished with correct placement, reward=7.476167863061919, evaluation=0.017891300571428573
Finished with correct placement, reward=7.930080771543647, evaluation=0.01590174476190476
Finished with correct placement, reward=8.99408082057243, evaluation=0.012361934222222223
Finished with correct placement, reward=7.308511048470638, evaluation=0.018721566476190478
Finished with correct placement, reward=7.262013720216386, evaluation=0.01896207542857143
Finished with correct placement, reward=6.94984329291416, evaluation=0.020703795809523813
Finished with correct placement, reward=7.105503454507231, evaluation=0.019806616380952383
Finished with correct placement, reward=8.982706079773244, evaluation=0.012393261714285715
Finished with correct placement, reward=7.360423042808606, evaluation=0.018458416761904763
Finished with correct placement, reward=5.602776768060063, evaluation=0.03185615542857143
Finished with correct placement, reward=8.146642487055873, evaluation=0.01506755149206349
Finis

Finished with correct placement, reward=5.679743699549066, evaluation=0.03099863161904762
Finished with correct placement, reward=7.100943258787344, evaluation=0.019832064
Finished with correct placement, reward=6.954346412734639, evaluation=0.020676992
Finished with correct placement, reward=8.402037040524714, evaluation=0.014165464380952382
Finished with correct placement, reward=6.339283624909925, evaluation=0.024883968000000003
Finished with correct placement, reward=6.339957640919966, evaluation=0.024878677333333335
Finished with correct placement, reward=6.957268517262237, evaluation=0.02065962666666667
19100: pos=20/22, reward=-5, evaluation=0.030951402666666666
%ok: 0.11
rolling average score of ok = 0.021381327714285716
tensor([[ 0.0283, -0.0252,  0.0445,  ..., -0.1175, -0.1067,  0.1275],
        [-0.1155, -0.0796, -0.0541,  ...,  0.1192, -0.0868, -0.0249],
        [ 0.0858,  0.0730, -0.0261,  ..., -0.1321, -0.1109,  0.0802],
        ...,
        [ 0.0297, -0.1056,  0.0836,  .

Finished with correct placement, reward=8.53601814488195, evaluation=0.013724272761904762
Finished with correct placement, reward=8.308565882437522, evaluation=0.014485979428571428
Finished with correct placement, reward=8.402370571892515, evaluation=0.01416433980952381
Finished with correct placement, reward=5.602085893231545, evaluation=0.031864013206349205
Finished with correct placement, reward=8.304793669078256, evaluation=0.014499142095238095
Finished with correct placement, reward=6.949472476485405, evaluation=0.020706005333333336
Finished with correct placement, reward=7.112954802243361, evaluation=0.01976514031746032
Finished with correct placement, reward=5.67797299787969, evaluation=0.031017968761904765
Finished with correct placement, reward=7.480513636190739, evaluation=0.01787051885714286
Finished with correct placement, reward=8.159680669522983, evaluation=0.015019437714285714
Finished with correct placement, reward=6.946777104531615, evaluation=0.020722076444444446
Fini

Finished with correct placement, reward=6.575313201435563, evaluation=0.02312954615873016
Finished with correct placement, reward=6.4522500366914555, evaluation=0.02402025549206349
Finished with correct placement, reward=7.103311825989676, evaluation=0.019818840380952385
20500: pos=19/22, reward=-5, evaluation=0.015023363047619047
%ok: 0.03
rolling average score of ok = 0.019538961725829727
tensor([[ 0.0283, -0.0252,  0.0445,  ..., -0.1175, -0.1067,  0.1275],
        [-0.1155, -0.0796, -0.0541,  ...,  0.1192, -0.0868, -0.0249],
        [ 0.0858,  0.0730, -0.0261,  ..., -0.1321, -0.1109,  0.0802],
        ...,
        [ 0.0297, -0.1056,  0.0836,  ...,  0.1178, -0.1212,  0.0496],
        [-0.0336,  0.0863,  0.1104,  ...,  0.1329, -0.1047,  0.0087],
        [ 0.1369,  0.1569, -0.0164,  ...,  0.1131, -0.1186, -0.1278]])


KeyboardInterrupt: 

In [ ]:
print(best_policy.linear1.weight.data)
print(best_policy.num_actions)

layer_file = "ml_graph_vgg.txt"
mem_file = "vgg_new_memory_calc.txt"

c = Computation_Graph(layer_file,mem_file)


device_file = "dev_graph.txt"
d = Device_Graph(device_file)
#d.print_info()

#c.check_topo_sort()


e_another = Environment(c,d)

cur_state = e_another.reset()
#e_another.print_current_state()



for i in range(0,22):
    print(i)
    temp_state = cur_state.copy()

    for p in range(4):
        temp_state[0][p] = float(temp_state[0][p])/MAX_OP_PARAM[p]

    for d in range(1,1+device_num):
        for p in range(4):
            temp_state[d][p] = float(temp_state[d][p])/MAX_ED_PARAM[p]

    action, log_prob = best_policy.get_action(temp_state.flatten())
    #print(action)
    cur_state, mapping, reward, done = e_another.step(action)
    #e_another.print_current_state()

#print(mapping)
mapping_file = "mapping_default.txt"

f = open(mapping_file,"w")

for key in mapping:
    f.write(c.layer_idx_to_name_map[key]+' '+str(mapping[key])+'\n')
f.close()

print('done')